# Necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS

2023-05-11 15:04:11.484517: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 15:04:11.484536: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df=pd.read_csv("3_imputed_data_for_cleaning.csv")
df=df.drop(['Unnamed: 0'],axis=1)
df

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic
1726830,10219057,2023-01-09,2,4.16,5.59,73.30,44.600000,3.0,23.2544,M,White,Non-Hispanic
1726831,10225856,2023-01-10,1,1.36,0.72,84.90,19.700000,3.0,0.9792,M,White,Non-Hispanic
1726832,10223584,2023-01-11,1,4.18,3.56,42.53,34.500000,1.0,14.8808,F,White,Non-Hispanic


In [3]:
list_unique_patients=df["PatientID"].unique().tolist()
print("Number of unique patients in the dataset",len(list_unique_patients))

Number of unique patients in the dataset 7399


In [4]:
#defining the dataframe
Wound_assessment=df

#defining the list which has all the unique patients' PatientID
list_patients=Wound_assessment["PatientID"].unique()

#defining to list the patients' index who will be used for forecasting and who will not be used for forecasting
patients_for_forecasting=[]
patients_not_for_forecasting=[]

#defining a variable where we want to count and store the number of total unique wounds for which we want to do the forecasting
unique_wound_number=0

for i in range(len(list_patients)):
    #print(i)
#     if i>9:
#         break
    patient=list_patients[i]
    df_patient=Wound_assessment.query("PatientID == @patient")
    if df_patient.shape[0]>1:
        patients_for_forecasting.append(list_patients[i])
        #print(i)
        wounds_of_list_patient=df_patient["WoundNumber"].unique().tolist()
        #print(wounds_of_list_patient)
        indivi_wound=0
        for j in range(len(wounds_of_list_patient)):
            wound_number= wounds_of_list_patient[j]
            df_patient_1_wound=df_patient.query("WoundNumber==@wound_number")
            #display(df_patient_1_wound)
            if df_patient_1_wound.shape[0]>1:
                if df_patient_1_wound.dropna().shape[0]>=1:
                    indivi_wound=indivi_wound+1
        print("For patient",i+1,", number of unique wounds for which forecasting can be done:",indivi_wound)
#                     #

#                     w_length=pd.Series(df_patient_1_wound["Length"].values)
#                     y = w_length.dropna().values
#                     x=w_length.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_length.index.values
#                     y1=cs(w_length.index.values)
#                     df_patient_1_wound["Length"]=y1


#                     w_width=pd.Series(df_patient_1_wound["Width"].values)
#                     y = w_width.dropna().values
#                     x=w_width.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_width.index.values
#                     y1=cs(w_width.index.values)
#                     df_patient_1_wound["Width"]=y1
                    
#                     w_depth=pd.Series(df_patient_1_wound["Depth"].values)
#                     y = w_depth.dropna().values
#                     x=w_depth.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_depth.index.values
#                     y1=cs(w_depth.index.values)
#                     df_patient_1_wound["Depth"]=y1
                    
                    
#                     display(df_patient_1_wound)
        #print("For patient",i,"number of unique wounds",len(wound_list_patient))
        #for j in range(df_patient_specific_wound=
        unique_wound_number=unique_wound_number+indivi_wound
        #list_count_patient_for_forecasting.append(i)
        #display(df_patient)
    else:
        patients_not_for_forecasting.append(list_patients[i])
        print("For patient",i+1,", Forecasting cannot be done because the patient has only one assessment")
print("Total unique wounds:",unique_wound_number)
print("Patients who have more than 1 assessments:",len(patients_for_forecasting))
print("Patients who have just 1 assessment:",len(patients_not_for_forecasting))

For patient 1 , number of unique wounds for which forecasting can be done: 6
For patient 2 , number of unique wounds for which forecasting can be done: 8
For patient 3 , number of unique wounds for which forecasting can be done: 1
For patient 4 , number of unique wounds for which forecasting can be done: 11
For patient 5 , number of unique wounds for which forecasting can be done: 15
For patient 6 , number of unique wounds for which forecasting can be done: 2
For patient 7 , number of unique wounds for which forecasting can be done: 9
For patient 8 , number of unique wounds for which forecasting can be done: 3
For patient 9 , number of unique wounds for which forecasting can be done: 5
For patient 10 , number of unique wounds for which forecasting can be done: 7
For patient 11 , number of unique wounds for which forecasting can be done: 3
For patient 12 , number of unique wounds for which forecasting can be done: 19
For patient 13 , number of unique wounds for which forecasting can be 

For patient 128 , number of unique wounds for which forecasting can be done: 21
For patient 129 , number of unique wounds for which forecasting can be done: 2
For patient 130 , number of unique wounds for which forecasting can be done: 9
For patient 131 , number of unique wounds for which forecasting can be done: 10
For patient 132 , number of unique wounds for which forecasting can be done: 11
For patient 133 , number of unique wounds for which forecasting can be done: 1
For patient 134 , number of unique wounds for which forecasting can be done: 2
For patient 135 , number of unique wounds for which forecasting can be done: 10
For patient 136 , number of unique wounds for which forecasting can be done: 9
For patient 137 , number of unique wounds for which forecasting can be done: 4
For patient 138 , number of unique wounds for which forecasting can be done: 6
For patient 139 , number of unique wounds for which forecasting can be done: 0
For patient 140 , number of unique wounds for wh

For patient 234 , number of unique wounds for which forecasting can be done: 5
For patient 235 , number of unique wounds for which forecasting can be done: 1
For patient 236 , number of unique wounds for which forecasting can be done: 1
For patient 237 , number of unique wounds for which forecasting can be done: 5
For patient 238 , number of unique wounds for which forecasting can be done: 13
For patient 239 , number of unique wounds for which forecasting can be done: 3
For patient 240 , number of unique wounds for which forecasting can be done: 7
For patient 241 , number of unique wounds for which forecasting can be done: 3
For patient 242 , number of unique wounds for which forecasting can be done: 4
For patient 243 , number of unique wounds for which forecasting can be done: 2
For patient 244 , number of unique wounds for which forecasting can be done: 4
For patient 245 , number of unique wounds for which forecasting can be done: 1
For patient 246 , number of unique wounds for which

For patient 348 , number of unique wounds for which forecasting can be done: 3
For patient 349 , number of unique wounds for which forecasting can be done: 12
For patient 350 , number of unique wounds for which forecasting can be done: 1
For patient 351 , number of unique wounds for which forecasting can be done: 2
For patient 352 , number of unique wounds for which forecasting can be done: 2
For patient 353 , number of unique wounds for which forecasting can be done: 0
For patient 354 , number of unique wounds for which forecasting can be done: 3
For patient 355 , number of unique wounds for which forecasting can be done: 1
For patient 356 , number of unique wounds for which forecasting can be done: 2
For patient 357 , number of unique wounds for which forecasting can be done: 1
For patient 358 , number of unique wounds for which forecasting can be done: 4
For patient 359 , number of unique wounds for which forecasting can be done: 1
For patient 360 , number of unique wounds for which

For patient 477 , number of unique wounds for which forecasting can be done: 3
For patient 478 , number of unique wounds for which forecasting can be done: 1
For patient 479 , number of unique wounds for which forecasting can be done: 1
For patient 480 , number of unique wounds for which forecasting can be done: 2
For patient 481 , number of unique wounds for which forecasting can be done: 1
For patient 482 , number of unique wounds for which forecasting can be done: 3
For patient 483 , number of unique wounds for which forecasting can be done: 5
For patient 484 , number of unique wounds for which forecasting can be done: 4
For patient 485 , number of unique wounds for which forecasting can be done: 1
For patient 486 , number of unique wounds for which forecasting can be done: 1
For patient 487 , number of unique wounds for which forecasting can be done: 1
For patient 488 , number of unique wounds for which forecasting can be done: 1
For patient 489 , number of unique wounds for which 

...

For patient 6845 , number of unique wounds for which forecasting can be done: 2
For patient 6846 , number of unique wounds for which forecasting can be done: 2
For patient 6847 , number of unique wounds for which forecasting can be done: 2
For patient 6848 , number of unique wounds for which forecasting can be done: 1
For patient 6849 , number of unique wounds for which forecasting can be done: 5
For patient 6850 , Forecasting cannot be done because the patient has only one assessment
For patient 6851 , number of unique wounds for which forecasting can be done: 6
For patient 6852 , number of unique wounds for which forecasting can be done: 1
For patient 6853 , number of unique wounds for which forecasting can be done: 1
For patient 6854 , number of unique wounds for which forecasting can be done: 1
For patient 6855 , number of unique wounds for which forecasting can be done: 2
For patient 6856 , number of unique wounds for which forecasting can be done: 2
For patient 6857 , number of u

For patient 6950 , number of unique wounds for which forecasting can be done: 1
For patient 6951 , number of unique wounds for which forecasting can be done: 1
For patient 6952 , number of unique wounds for which forecasting can be done: 2
For patient 6953 , number of unique wounds for which forecasting can be done: 3
For patient 6954 , number of unique wounds for which forecasting can be done: 5
For patient 6955 , number of unique wounds for which forecasting can be done: 1
For patient 6956 , number of unique wounds for which forecasting can be done: 6
For patient 6957 , number of unique wounds for which forecasting can be done: 14
For patient 6958 , number of unique wounds for which forecasting can be done: 2
For patient 6959 , Forecasting cannot be done because the patient has only one assessment
For patient 6960 , number of unique wounds for which forecasting can be done: 2
For patient 6961 , number of unique wounds for which forecasting can be done: 3
For patient 6962 , number of 

For patient 7063 , number of unique wounds for which forecasting can be done: 8
For patient 7064 , number of unique wounds for which forecasting can be done: 1
For patient 7065 , number of unique wounds for which forecasting can be done: 3
For patient 7066 , number of unique wounds for which forecasting can be done: 1
For patient 7067 , number of unique wounds for which forecasting can be done: 2
For patient 7068 , number of unique wounds for which forecasting can be done: 1
For patient 7069 , number of unique wounds for which forecasting can be done: 12
For patient 7070 , number of unique wounds for which forecasting can be done: 1
For patient 7071 , number of unique wounds for which forecasting can be done: 1
For patient 7072 , number of unique wounds for which forecasting can be done: 1
For patient 7073 , number of unique wounds for which forecasting can be done: 1
For patient 7074 , number of unique wounds for which forecasting can be done: 1
For patient 7075 , number of unique wou

For patient 7176 , number of unique wounds for which forecasting can be done: 1
For patient 7177 , number of unique wounds for which forecasting can be done: 6
For patient 7178 , number of unique wounds for which forecasting can be done: 10
For patient 7179 , number of unique wounds for which forecasting can be done: 1
For patient 7180 , number of unique wounds for which forecasting can be done: 1
For patient 7181 , number of unique wounds for which forecasting can be done: 1
For patient 7182 , number of unique wounds for which forecasting can be done: 1
For patient 7183 , number of unique wounds for which forecasting can be done: 1
For patient 7184 , number of unique wounds for which forecasting can be done: 1
For patient 7185 , number of unique wounds for which forecasting can be done: 1
For patient 7186 , number of unique wounds for which forecasting can be done: 1
For patient 7187 , number of unique wounds for which forecasting can be done: 3
For patient 7188 , number of unique wou

For patient 7288 , number of unique wounds for which forecasting can be done: 1
For patient 7289 , number of unique wounds for which forecasting can be done: 1
For patient 7290 , Forecasting cannot be done because the patient has only one assessment
For patient 7291 , number of unique wounds for which forecasting can be done: 5
For patient 7292 , number of unique wounds for which forecasting can be done: 2
For patient 7293 , number of unique wounds for which forecasting can be done: 1
For patient 7294 , number of unique wounds for which forecasting can be done: 1
For patient 7295 , number of unique wounds for which forecasting can be done: 7
For patient 7296 , number of unique wounds for which forecasting can be done: 4
For patient 7297 , Forecasting cannot be done because the patient has only one assessment
For patient 7298 , Forecasting cannot be done because the patient has only one assessment
For patient 7299 , number of unique wounds for which forecasting can be done: 2
For patien

### So we have 370 patients for whom we don't have more than 1 wound assessments, thus we will remove those patients' specific wound information from the original dataframe, or better make two new dataframes

In [5]:
df_non=pd.DataFrame()
for i in range(len(patients_not_for_forecasting)):
    #print(i)
    patient=patients_not_for_forecasting[i]
    df_not_for_forecasting=Wound_assessment.query("PatientID == @patient")
    #display(df_not_for_forecasting)
    df_non=pd.concat([df_non,df_not_for_forecasting],axis=0)
df_non=df_non.reset_index()
df_non=df_non.drop(["index"],axis=1)
df_non

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,6223300,2017-03-07,1,0.60,0.40,61.80,24.300000,0.0,0.2400,M,White,Non-Hispanic
1,6193541,2017-03-16,1,1.00,0.80,86.70,20.800000,4.0,0.8000,F,White,Non-Hispanic
2,6227686,2017-03-21,1,0.70,0.70,80.72,21.000000,4.0,0.4900,F,White,Non-Hispanic
3,6246709,2017-03-21,1,9.00,1.50,5.67,34.264272,4.0,13.5000,M,White,Non-Hispanic
4,6364023,2017-05-10,1,7.00,6.00,82.62,26.000000,3.0,42.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
365,10223596,2023-01-04,1,0.93,0.61,79.02,26.600000,1.0,0.5673,F,White,Non-Hispanic
366,10218464,2023-01-05,1,8.08,4.29,92.10,17.200000,4.0,34.6632,F,White,Non-Hispanic
367,10225856,2023-01-10,1,1.36,0.72,84.90,19.700000,3.0,0.9792,M,White,Non-Hispanic
368,10223584,2023-01-11,1,4.18,3.56,42.53,34.500000,1.0,14.8808,F,White,Non-Hispanic


### Check for value_counts of patients for whom we cannot do the forecasting

In [6]:
print(df_non["PatientGender_new"].value_counts(),"\n")
print(df_non["RaceCodeList_new"].value_counts(),"\n")
print(df_non["patientEthnicity_new"].value_counts())

M    200
F    170
Name: PatientGender_new, dtype: int64 

White    338
Black     24
ASKU       5
Asian      2
Other      1
Name: RaceCodeList_new, dtype: int64 

Non-Hispanic    361
Hispanic          9
Name: patientEthnicity_new, dtype: int64


### Then we have 7019 patients for whom we have more than 1 wound assessments, thus we will create a separate dataframe using those patients' information from the original dataframe, and use that for further analysis

In [7]:
#df_for=pd.DataFrame()
indexes_to_remove=[]
for i in range(len(patients_not_for_forecasting)):
    #print(i)
    patient=patients_not_for_forecasting[i]
    df_not_for_forecasting=Wound_assessment.query("PatientID == @patient")
    indexes_to_remove.append(int(df_not_for_forecasting.index.values))
    #display(df_not_for_forecasting)
#     df_non=pd.concat([df_non,df_not_for_forecasting],axis=0)
# df_non=df_non.reset_index()
# df_non=df_non.drop(["index"],axis=1)
indexes_to_remove

[73265,
 87024,
 88062,
 88063,
 110912,
 115736,
 116266,
 123669,
 138857,
 139709,
 143516,
 178929,
 196307,
 197807,
 203543,
 204699,
 206937,
 207595,
 207731,
 207779,
 208237,
 208839,
 209156,
 209354,
 233671,
 236571,
 241159,
 241276,
 241380,
 241640,
 241641,
 255763,
 266064,
 266194,
 266285,
 266312,
 268214,
 268215,
 274395,
 274396,
 281253,
 289423,
 289460,
 291298,
 295375,
 318728,
 318797,
 320952,
 323069,
 335529,
 335562,
 335905,
 335906,
 336893,
 337122,
 338312,
 338409,
 344617,
 344749,
 350154,
 350278,
 360135,
 362449,
 362605,
 362606,
 364190,
 364207,
 369883,
 371606,
 376156,
 385831,
 386634,
 391722,
 392446,
 392686,
 398079,
 398322,
 399460,
 401823,
 407547,
 408611,
 408620,
 411331,
 413027,
 414162,
 414163,
 414636,
 414757,
 422663,
 422674,
 426206,
 428732,
 428940,
 430805,
 430836,
 430837,
 436048,
 436141,
 441240,
 441641,
 442074,
 442938,
 444508,
 444761,
 445394,
 447592,
 452411,
 452450,
 453238,
 453291,
 453396,
 4548

In [8]:
df_for=Wound_assessment
df_for=df_for.drop(indexes_to_remove,axis=0)
df_for=df_for.reset_index()
df_for=df_for.drop(["index"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1726459,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1726460,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1726461,10219057,2023-01-09,1,1.44,0.84,73.30,44.600000,3.0,1.2096,M,White,Non-Hispanic
1726462,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


In [9]:
df_for.to_csv("df_for.csv")

In [10]:
df_for=pd.read_csv("df_for.csv")
df_for=df_for.drop(["Unnamed: 0"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1726459,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1726460,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1726461,10219057,2023-01-09,1,1.44,0.84,73.30,44.600000,3.0,1.2096,M,White,Non-Hispanic
1726462,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


In [11]:
df_for_squeezed=df_for.drop_duplicates(subset='PatientID', keep="first")
print(df_for_squeezed["PatientGender_new"].value_counts(),"\n")
print(df_for_squeezed["RaceCodeList_new"].value_counts(),"\n")
print(df_for_squeezed["patientEthnicity_new"].value_counts())

M    3641
F    3388
Name: PatientGender_new, dtype: int64 

White                                        6292
Black                                         684
Asian                                          24
ASKU                                           19
American Indian or Alaska Native                6
Native Hawaiian or Other Pacific Islander       3
Other                                           1
Name: RaceCodeList_new, dtype: int64 

Non-Hispanic        6937
Hispanic              83
Patient Declined       8
ASKU                   1
Name: patientEthnicity_new, dtype: int64


### Now that we have a dataframe of 7019 patients who have more than one assessments, we want to remove those patients also who have all NULL information in the assessments, like all assessments are NULL

### Now checking for individual wounds per patients and checking if there is any wound for which we have only one assessment. If yes, then we are removing it.

In [12]:
list_patient=df_for["PatientID"].unique().tolist()

list_index_wound_specific_1_assessment=[]
for i in range(len(list_patient)):
    print(i)
    patient=list_patient[i]
    df_indivi_patient_all_wounds=df_for.query("PatientID == @patient")
    list_wounds=df_indivi_patient_all_wounds["WoundNumber"].unique().tolist()
    for j in range(len(list_wounds)):
        woundnumber=list_wounds[j]
        df_for_forecasting=df_indivi_patient_all_wounds.query("WoundNumber == @woundnumber")
        #display(df_for_forecasting)
        if df_for_forecasting.shape[0]==1:
            list_index_wound_specific_1_assessment.append(int(df_for_forecasting.index.values))
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091


3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774


5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436


6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028


In [13]:
print("For",len(list_index_wound_specific_1_assessment),"number of unique wounds we don't have more than 1 assessments, thus we will remove these information from the original dataframe.")

For 798 number of unique wounds we don't have more than 1 assessments, thus we will remove these information from the original dataframe.


In [14]:
df_for=df_for.drop(list_index_wound_specific_1_assessment,axis=0)
df_for=df_for.reset_index()
df_for=df_for.drop(["index"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1725661,10210244,2023-01-05,1,0.94,0.84,87.58,34.264272,3.0,0.7896,M,White,Non-Hispanic
1725662,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1725663,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1725664,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


### Now removing those information if for specific patient, specific wound, all assessments are NULL

In [15]:
list_patient=df_for["PatientID"].unique().tolist()

list_index_wound_specific_all_null_assessment=[]
for i in range(len(list_patient)):
    print(i)
    patient=list_patient[i]
    df_indivi_patient_all_wounds=df_for.query("PatientID == @patient")
    list_wounds=df_indivi_patient_all_wounds["WoundNumber"].unique().tolist()
    for j in range(len(list_wounds)):
        woundnumber=list_wounds[j]
        df_for_forecasting=df_indivi_patient_all_wounds.query("WoundNumber == @woundnumber")
        #df_for_forecasting_1=df_for_forecasting.drop(["Depth"],axis=1)
        df_for_forecasting_1=df_for_forecasting.dropna()
        if df_for_forecasting_1.shape[0]<1:
            display(df_for_forecasting_1)
            for k in range(len(df_for_forecasting.index.values)):
                list_index_wound_specific_all_null_assessment.append(df_for_forecasting.index.values[k])
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089


3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737


5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384


In [16]:
df_for=df_for.drop(list_index_wound_specific_all_null_assessment,axis=0)
df_for=df_for.reset_index()
df_for=df_for.drop(["index"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1725661,10210244,2023-01-05,1,0.94,0.84,87.58,34.264272,3.0,0.7896,M,White,Non-Hispanic
1725662,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1725663,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1725664,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


In [17]:
df_for.to_csv("4_df_cleaned_for_outlier_removal.csv")

In [18]:
df_for["PatientID"].nunique()

6856

In [19]:
wounds=0
list_patients=df_for["PatientID"].unique().tolist()
for i in range(len(list_patients)):
    print(i)
    patient=list_patients[i]
    data_1=df_for.query("PatientID==@patient")
    number=data_1["WoundNumber"].nunique()
    wounds=wounds+number
print("Number of total unique wounds",wounds)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066


3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776


5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485


In [20]:
print(df["PatientGender_new"].value_counts(),"\n")
print(df["RaceCodeList_new"].value_counts(),"\n")
print(df["patientEthnicity_new"].value_counts())

M    998770
F    728064
Name: PatientGender_new, dtype: int64 

White                                        1604337
Black                                         119817
ASKU                                            1564
American Indian or Alaska Native                 612
Asian                                            472
Native Hawaiian or Other Pacific Islander         22
Other                                             10
Name: RaceCodeList_new, dtype: int64 

Non-Hispanic        1716930
Hispanic               9733
Patient Declined        155
ASKU                     16
Name: patientEthnicity_new, dtype: int64
